<a href="https://colab.research.google.com/github/Satyake/Getting-Started-with-TF2/blob/main/Model_SubClassing_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout, Softmax, concatenate
import tensorflow as tf

In [ ]:
class My_model(Model):
  def __init__(self):
    super(My_model,self).__init__()
    self.dense_1=Dense(64,activation='relu')
    self.dense_2=Dense(10)

  def call(self, inputs):
    x=self.dense_1(inputs)
    return self.dense_2(x)

In [ ]:
model=My_model()
model(tf.random.uniform([1,10]))
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  704       
                                                                 
 dense_1 (Dense)             multiple                  650       
                                                                 
Total params: 1,354
Trainable params: 1,354
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class My_model(Model):
  def __init__(self):
    super(My_model,self).__init__()
    self.dense_1=Dense(64,activation='relu')
    self.dense_2=Dense(10)
    self.dense_3=Dense(5)
    self.dropout=Dropout(0.4)
    self.softmax=Softmax()

  def call(self, inputs,training=True):
    x=self.dense_1(inputs)
    y1=self.dense_2(inputs)
    y2=self.dense_3(y1)
    concat=concatenate([x,y2])
    return self.softmax(concat)

In [ ]:
model=My_model()
model(tf.random.uniform([1,10]))
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             multiple                  704       
                                                                 
 dense_3 (Dense)             multiple                  110       
                                                                 
 dense_4 (Dense)             multiple                  55        
                                                                 
 dropout (Dropout)           multiple                  0 (unused)
                                                                 
 softmax (Softmax)           multiple                  0         
                                                                 
Total params: 869
Trainable params: 869
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Softmax

In [ ]:
class My_layer(Layer):
  def __init__(self,units,input_dim):
    super(My_layer,self).__init__()
    self.w=self.add_weight(shape=(input_dim,units),initializer='random_normal')
    self.b=self.add_weight(shape=(units),initializer='zeros')

    def call(self,inputs):
      return tf.matmul(inputs,self.w)+self.b



In [ ]:
dense_layer=My_layer(3,5)
x=tf.ones((1,5))
print(dense_layer(x))
print(dense_layer.weights)

tf.Tensor([[1. 1. 1. 1. 1.]], shape=(1, 5), dtype=float32)
[<tf.Variable 'Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[-0.04148162, -0.03699851, -0.04818638],
       [-0.11452555, -0.01751169, -0.02340675],
       [ 0.044606  , -0.04807325,  0.03252507],
       [ 0.04567571,  0.04104174,  0.04394555],
       [-0.05608606, -0.00826648, -0.07517032]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


In [ ]:
#setting trainable to false for freezing and unfreezing of weights
print(len(dense_layer.trainable_weights))
print(len(dense_layer.non_trainable_weights))

2
0


In [ ]:
# setting non trainablility 
class My_layer(Layer):
  def __init__(self,units,input_dim):
    super(My_layer,self).__init__()
    self.w=self.add_weight(shape=(input_dim,units),initializer='random_normal',trainable=False)
    self.b=self.add_weight(shape=(units),initializer='zeros')

    def call(self,inputs):
      return tf.matmul(inputs,self.w)+self.b



In [ ]:
#create custom layer to accumulate means of output values
class Layer_Accumulation(Layer):
  def __init__(self,units,input_dim):
    super(Layer_Accumulation,self).__init__()
    self.w=self.add_weight(shape=(input_dim,units),trainable=True)
    self.b=self.add_weight(shape=(units,),trainable=True)
    self.sum_act=tf.Variable(initial_value=tf.zeros((units)),trainable=False)
    self.num_cnt=tf.Variable(initial_value=0,trainable=False)
  def call(self,inputs):
    output=tf.matmul(inputs,self.w)+self.b
    self.sum_act.assign_add(tf.reduce_sum(output,axis=0))
    self.num_cnt.assign_add(inputs.shape[0])
    return output, self.num_cnt/(self.num_cnt)



    

In [ ]:
#Testing the layer
dense=Layer_Accumulation(3,5)
y, activation_means=dense(tf.ones((1,5)))


In [ ]:
print(y)

tf.Tensor([[ 0.40888172 -0.37048787 -1.1715807 ]], shape=(1, 3), dtype=float32)


In [ ]:
print(dense.trainable_weights)

[<tf.Variable 'Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[-0.3516118 , -0.8073976 , -0.12117243],
       [ 0.8551691 , -0.09330612, -0.22985095],
       [-0.71801496, -0.44855377,  0.16776055],
       [ 0.40865988, -0.37891626, -0.49132848],
       [-0.530041  ,  0.8344491 , -0.3900732 ]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([ 0.74472046,  0.52323675, -0.10691619], dtype=float32)>]


In [ ]:
print(activation_means.numpy())

1.0


In [ ]:
#Dropouts
class mydropout(Layer):
  def __init__(self,rate):
    super(mydropout,self).__init__()
    self.rate=rate
  def call(self,inputs):
    return tf.nn.dropout(inputs,rate=self.rate)

In [ ]:
class complete_model(Model):
  def __init__(self,units_1,input_dim1,units_2,units_3):
    super(complete_model,self).__init__()
    self.layer1= My_layer(units_1,input_dim1)
    self.dropout1=mydropout(0.5)
    self.layer2= My_layer(units_2,units_1)
    self.dropout2=mydropout(0.5)
    self.layer3= My_layer(units_3,units_2)
    self.softmax=Softmax()
  def call(self,inputs):
    x=self.layer1(inputs)
    x=tf.nn.relu(x)
    x=self.dropout1(x)
    x=self.layer2(x)
    x=tf.nn.relu(x)
    x=self.dropout2(x)
    x= self.softmax(x)
    return x

In [ ]:
model=complete_model(64,1000,64,46)
input_shape=(64,1000,64,46)
model.build(input_shape)
#print(model(tf.ones((1,1000))))
model.summary()

Model: "complete_model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_layer_22 (My_layer)      multiple                  64064     
                                                                 
 mydropout_14 (mydropout)    multiple                  0         
                                                                 
 my_layer_23 (My_layer)      multiple                  4160      
                                                                 
 mydropout_15 (mydropout)    multiple                  0         
                                                                 
 my_layer_24 (My_layer)      multiple                  0 (unused)
                                                                 
 softmax_8 (Softmax)         multiple                  0         
                                                                 
Total params: 71,214
Trainable params: 71,214
Non-